In [ ]:
import cudf as pd
import cupy as np
import os
import cv2
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.metrics.confusion_matrix import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
import random

In [ ]:
#3875-pneumonia train
#1341-normal train

In [ ]:
#as we know number of pnemonia samples are way more than normal so we have to eaqual that number

In [ ]:
image_size = 200
labels = ['PNEUMONIA', 'NORMAL']

def data_loader(data_dir):
    data = list()
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img), 0)
            resized_arr = cv2.resize(img_arr, (image_size, image_size))
            data.append([resized_arr, class_num]) 
    return data

In [ ]:
val = data_loader('../input/chest-xray-pneumonia/chest_xray/val')
test = data_loader('../input/chest-xray-pneumonia/chest_xray/test')
train = data_loader('../input/chest-xray-pneumonia/chest_xray/train')

In [ ]:
def takeSecond(elem):
    return elem[1]
train.sort(key=takeSecond,reverse = True)

for i in range(20):
    print(train[i][1])

In [ ]:
train=train[:2682][:]

In [ ]:
train[0][1]

In [ ]:
random.shuffle(train)
random.shuffle(test)
random.shuffle(val)

In [ ]:
for i in range(5):
    plt.figure(figsize = (3,3))
    plt.imshow(np.asnumpy(train[i][0]))
    plt.title(labels[train[i][1]])


In [ ]:
#normalize
def normalize_list(train):
    for pair in train:
        pair[0] = np.array(pair[0])/ 255
    return train

train = normalize_list(train)
test = normalize_list(test)
val = normalize_list(val)

In [ ]:
print(train[4][1])
train[4][0]

In [ ]:
#splitting data and target from train,test and val
def make_x_y(data):
    X = []
    Y = []
    for pair in data:
        X.append(pair[0])
        Y.append(pair[1])
    return np.array(X),np.array(Y)
x_train, y_train = make_x_y(train)
x_test, y_test = make_x_y(test)
x_val, y_val = make_x_y(val)

In [ ]:
plt.figure(figsize = (3,3))
plt.imshow(np.asnumpy(x_train[4]))

In [ ]:
count1=0
count0=0
for i in range(len(y_train)):
    if y_train[i]==0:
        count0=count0+1
    else:
        count1=count1+1

print(count0)
print(count1)

In [ ]:
#converting image from 2-d to 1-d array
x_train = x_train.reshape(len(x_train), 200*200)
x_test = x_test.reshape(len(x_test), 200*200)
x_val = x_val.reshape(len(x_val), 200*200)

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

In [ ]:
from cuml.linear_model import LogisticRegression
from cuml.multiclass import MulticlassClassifier
from cuml.multiclass import OneVsRestClassifier

In [ ]:
#OneVsRestClassifier

In [ ]:
cls = OneVsRestClassifier(LogisticRegression())
cls.fit(x_train.astype('float32'),y_train.astype('float32'))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix


In [ ]:
y_pred = cls.predict(x_test.astype('float32'))
y_pred


In [ ]:
type(y_pred)

In [ ]:
accuracy_score(np.asnumpy(y_test),np.asnumpy(y_pred))


In [ ]:
#Using Random forest

In [ ]:
#random forest classifier model
cuml_model = cuRFC(max_features=1.0,n_bins=8, n_estimators=40)
# making the model and fitting the training data on the model
cuml_model.fit(x_train.astype('float32'), y_train.astype('float32'))

In [ ]:
y_rfc_predict = cuml_model.predict(x_test.astype('float32'))
print(type(y_rfc_predict))
print(y_rfc_predict.shape)

In [ ]:
y_test_temp = y_test.reshape(1, len(y_test))[0]
print(type(y_test_temp))
print(y_test_temp.shape)

In [ ]:
y_np_test = np.asnumpy(y_test_temp)
y_np_rfc_predict = np.asnumpy(y_rfc_predict)
print(type(y_np_test))
print(type(y_np_rfc_predict))

In [ ]:
accu = accuracy_score(y_np_test, y_np_rfc_predict)
print("accuracy of model is: %f" %accu)

In [ ]:
#using CNN

In [ ]:
# Import Tensorflow and check version
import tensorflow as tf
# Import TensorFlow Keras
from tensorflow import keras

In [ ]:
x_train[0].shape

In [ ]:
simple_model = keras.Sequential([
    # Flatten two dimansional images into one dimansion 200*200pixles=40000pixels.
    keras.layers.Input(shape=(40000,)),
    # First dense/ fully connected layer: 128 nodes.
    keras.layers.Dense(128, activation='relu'),
    # Second dense/ fully connected layer: 10 nodes --> Result is a score for each images class.
    keras.layers.Dense(2)])

In [ ]:
simple_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
simple_model.fit(np.asnumpy(x_train), np.asnumpy(y_train), epochs=15)

In [ ]:
test_loss, test_acc = simple_model.evaluate(np.asnumpy(x_test),np.asnumpy(y_test) )

In [ ]:
#so we can see random forest gives us 84% accurate solution


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
accuracy  = accuracy_score(y_np_test, y_np_rfc_predict)
precision  = precision_score(y_np_test, y_np_rfc_predict)
recall  = recall_score(y_np_test, y_np_rfc_predict)
f1  = f1_score(y_np_test, y_np_rfc_predict)
roc  = roc_auc_score(y_np_test, y_np_rfc_predict)

value = [accuracy, precision, recall, f1, roc]
labels = ['Accuarcy', 'Precision', 'Recall', 'F1', 'ROC Score']

plt.figure(figsize = (6, 4), dpi=150)
plt.bar(labels, value)
plt.title('Metrics')
plt.show()

In [ ]:
pred = cuml_model.predict(x_test)
pred = np.asnumpy(pred)

fpr, tpr, threshold = metrics.roc_curve(y_np_test, pred)
roc_auc = metrics.auc(fpr, tpr)

plt.figure(figsize=(6, 4), dpi = 150)
plt.title('ROC curve')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
print("f1 score : %f"%f1)
print("accuracy is : %f"%(accu*100))
print("recall :%f"%recall)